# Compressing the System

## Objectives

### Questions

* How do I compress the system to a target density? 
* What is a packing fraction?

### Objectives

* Show how to compute the **packing fraction** of a system.
* Explain how how an **Updater** is an **operation** that modifies the system when its **Trigger** returns `True`.
* Demonstrate using the **QuickCompress** updater to achieve a target packing fraction.
* Show how some **operations** can stop the simulation **run**.

In [1]:
import hoomd
import math
import copy

## Packing fraction

Self-assembly in the [hard regular tetrahedra system](https://doi.org/10.1038/nature08641) occurs at a **packing fraction** of 0.55.
The **packing fraction** is the ratio of the volume occupied by the particles to the volume of the **periodic box**.
So far in this tutorial, we have **randomized** a system of *N* tetrahedra in a box with a very low packing fraction and stored that in `random.gsd`.
Let's initialize a **Simulation** with this configuration and see what packing fraction it is at.

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
sim.create_state_from_gsd(filename='random.gsd')

HOOMD-blue v2.9.0-1828-gd903f0dc6 GPU [CUDA] (10.2) DOUBLE HPMC_MIXED SSE SSE2 
Compiled: 06/08/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
HOOMD-blue is running on the CPU


Compute the [volume of our tetrahedron](https://en.wikipedia.org/wiki/Tetrahedron):

In [3]:
a = math.sqrt(8/3)
V_particle = a**3 / (6 * math.sqrt(2))

Compute the **packing fraction** (TODO: Add number of particles to state?)

In [4]:
initial_packing_fraction = sim.state.snapshot.particles.N * V_particle / sim.state.box.volume
print(initial_packing_fraction)

0.052991804257688346


As you can see, this **packing fraction** is very low and the box volume needs to be significantly reduced to achieve a **packing fraction** of 0.5.

We will use HPMC to move particles into non-overlapping configurations while we compress the system.
Set up the HPMC integrator for the tetrahedron simulations:

In [5]:
mc = hoomd.hpmc.integrate.ConvexPolyhedron(seed=42)
mc.shape['tetrahedron'] = dict(vertices=[(math.sqrt(8/9), 0, -1/3),
                                         (-math.sqrt(2/9), math.sqrt(2/3), -1/3),
                                         (-math.sqrt(2/9), -math.sqrt(2/3), -1/3),
                                         (0,0,1)])

sim.operations.integrator = mc

## The QuickCompress updater

An **Updater** is a type of **operation** in HOOMD-blue that makes changes to the **state**.
To use an **Updater**, we need to instantiate the object, assign a **Trigger**, and add it to the **Simulation**.
**Simulation** will apply the **Updater** on **time steps** where the **Trigger** is `True`.
The most commonly used **Trigger** is **PeriodicTrigger** which executes the **Updater** ever `period` steps.

**QuickCompress** is an **Updater** that works with HPMC to quickly compress the box to a target volume.
During compression, **QuickCompress** reduces the box volume by a scale factor, while *allowing slight overlaps between the particles*.
It then waits for the translation and rotation **trial moves** to remove these overlaps before it reduces the volume again.
This process temporarily produces invalid system configurations, but is much quicker than a process that does not allow temporary overlaps.

Compute the final box size with a **packing fraction** of 0.55 and configure a **QuickCompress** to **trigger** every 10 **time steps**.

In [6]:
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
final_packing_fraction = 0.55
final_box.volume = sim.state.snapshot.particles.N * V_particle / final_packing_fraction
# TODO: Implement QuickCompress
# compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10), target_box = final_box)

Add the **Updater** to the **Simulation**:

In [7]:
# sim.operations.updaters += compress

In [ ]:
# This code is not part of the tutorial. 
# It is a temporary Python implementation of QuickCompress method. Will re-implement in C++ as an Updater later.
D = 2
max_scale = 0.99
max_overlaps = 82*4 / 4
mc.d['tetrahedron'] = 0.02693819753092484
mc.a['tetrahedron'] = 0.019871213815167245

import time
import numpy
start_time = time.time()

current_box = hoomd.Box.from_box(initial_box)

N = sim.state.snapshot.particles.N
sim.operations.schedule()

import sys

count = 0

while final_box.volume < current_box.volume:
    
    v = numpy.random.random(1)
    current_scale = v * max_scale + (1-v)*1

    prev_volume = current_box.volume
    current_box.volume = current_scale * current_box.volume

    if current_box.volume < final_box.volume:
        current_box.volume = final_box.volume

    hoomd.update.BoxResize.scale_state(sim.state, current_box)

    overlaps = mc.overlaps

    if overlaps > max_overlaps:
        current_box.volume = prev_volume
        hoomd.update.BoxResize.scale_state(sim.state, current_box)
        overlaps = mc.overlaps
        
        sim.run(10)
       
#     if sim.timestep % 100 == 0:
#         print("phi =", (N*V_particle) / (current_box.volume), ": overlaps =", overlaps);
    
    # run until all overlaps are removed
    while overlaps > 0:
        sim.run(10);
        
        overlaps = mc.overlaps;
        sys.stdout.flush();
        
        gamma = 2.0
#         acceptance = mc.translate_moves[0] / sum(mc.translate_moves)
#         d_scale = ((1.0 + gamma) * acceptance) / (0.2 + gamma * acceptance)
#         mc.d['tetrahedron'] = mc.d['tetrahedron'] * d_scale
#         max_scale = (3*1 + (1 - mc.d['tetrahedron']/D)**3)/4

#         acceptance = mc.rotate_moves[0] / sum(mc.rotate_moves)
#         a_scale = ((1.0 + gamma) * acceptance) / (0.2 + gamma * acceptance)
#         mc.a['tetrahedron'] = mc.a['tetrahedron'] * a_scale

#print("phi =", (N*V_particle) / (current_box.volume), ": overlaps =", overlaps);
#end_time = time.time()
#print(end_time - start_time)

Let's save the final configuration to a GSD file for use in the next stage of the simulation.

In [9]:
hoomd.dump.GSD.write_state(state=sim.state, filename='compressed.gsd')